# Timeline Analysis

In [ ]:
import snap
from snap import TUNGraph
import os
import sys
import operator
import pandas as pd
import subprocess
import numpy as np
import csv
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import dates as mdates
import seaborn as sns
from __future__ import print_function
from IPython.display import HTML, display
import tabulate
import json
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from collections import Counter
import re

# Set Seaborn defaults
sns.set()
%matplotlib inline
pd.set_option("display.precision", 3)
mpl.rcParams['figure.dpi'] = 100
mpl.rcParams['savefig.dpi'] = 100
mpl.rcParams['figure.autolayout'] = True

# Global variables
data_dir = "../data"
pictures_path = os.path.join("../Pictures", "6.HashtagAnalysis")
tweets_path = "../lib/GetOldTweets-python/out/completed"

In [ ]:
def load_graph_from_backup(filename):
    FIn = snap.TFIn("../data/"+filename+".bin")
    graph = snap.TUNGraph.Load(FIn)
    return graph

def read_large_file(file_object):
    while True:
        data = file_object.readline()
        if not data:
            break
        yield data.rstrip('\n')
        
def get_usernames_from_basic_tweet_info(hashtag, tweet):
    usernames = set()
    # (1): Has tweeted using hashtag
    if hashtag in [h.lower() for h in tweet["entities"]["hashtags"]]:
        usernames.add(tweet["user"]["screen_name"].lower())

    # (2): Has been mentioned / replied to
    if not tweet["in_reply_to_screen_name"] is None:
        usernames.add(tweet["in_reply_to_screen_name"].lower())
    for mentions in tweet["entities"]["user_mentions"]:
        usernames.add(mentions["screen_name"].lower())
    return usernames

def extract_hashtag_usernames(hashtag, tweets):
    hashtag_usernames = set()
    for tweet in tweets:
        hashtag_usernames.update(get_usernames_from_basic_tweet_info(hashtag, tweet))
   
        # (3): Retweeted / Quoted status
        if "retweeted_status" in tweet:
            hashtag_usernames.update(get_usernames_from_basic_tweet_info(hashtag, tweet["retweeted_status"]))
        if "quoted_status" in tweet:
            hashtag_usernames.update(get_usernames_from_basic_tweet_info(hashtag, tweet["quoted_status"]))
    print("Total unique usernames involved in '#%s' hashtag conversations from %d tweets: %d" %(hashtag, len(tweets), len(hashtag_usernames)))
    return hashtag_usernames

In [ ]:
hashtag = "isis"
hashtag_full = "#ISIS"

In [ ]:
%%time
tweets_filename = os.path.join(tweets_path,"tweets_#" + hashtag + "_2013-09-01_2016-12-31.json")
tweets = []
with open(tweets_filename) as fin:
    for line in read_large_file(fin):
        tweets.append(json.loads(line))
print("Imported %d tweets from %s" %(len(tweets),tweets_filename))

In [ ]:
hashtag_usernames = extract_hashtag_usernames(hashtag, tweets)

In [ ]:
%%time
# First load all usernames in memory
usernames_to_id_dict = {}
with open("../data/usernames.csv") as usernames_f:
    for line in read_large_file(usernames_f):
        username = line.split(',')[0]
        encoding = int(line.split(',')[1])
        usernames_to_id_dict[username] = encoding